In [ ]:
#подключенные библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

#загрузка файла для работы
df=pd.read_csv("F:\\curs.csv")

df




# перевод данных в integer для наилудшего понимания машины
mapping = {'DEBIT': 1, 'TRANSFER': 2, 'CASH': 3, 'PAYMENT':4}
df['payment_type'] = df['payment_type'].map(mapping)
mapping2={'Europe': 1, 'LATAM': 2, 'USCA': 3, 'Pacific Asia':4,'Africa':5}
df['market'] = df['market'].map(mapping2)

mapping3 = {'Standard Class': 1, 'Second Class': 2, 'First Class': 3, 'Same Day':4}
df['shipping_mode'] = df['shipping_mode'].map(mapping3)
mapping4 = {'Puerto Rico': 1, 'EE. UU.': 2}
df['customer_country'] = df['customer_country'].map(mapping4)
mapping5 = {'Consumer': 1, 'Corporate': 2,'Home Office':3}
df['customer_segment'] = df['customer_segment'].map(mapping5)

mapping6 = {'PR':1, 'CA':2, 'KY':3, 'NJ':4, 'AZ':5, 'PA':6, 'NY':7, 'OH':8, 'CO':9, 'MT':10, 'WI':11, 'IL':12, 'DC':13, 'CT':14,
 'WV':15, 'UT':16, 'FL':17, 'TX':18, 'MI':19, 'NM':20, 'NV':21, 'WA':22, 'NC':23, 'GA':24, 'MD':25, 'SC':26, 'TN':27, 'IN':28,
 'MO':29, 'MN':30, 'OR':31, 'VA':32, 'MA':33, 'HI':34, 'RI':35, 'DE':36, 'ID':37, '91732':38, 'LA':39, 'ND':40, 'KS':41, 'IA':42,
 'OK':43, 'AL':44}
df['customer_state'] = df['customer_state'].map(mapping6)
mapping7 = {'Western Europe':1, 'South America':2, 'Central America':3, 'East of USA':4,
 'West of USA ':5, 'Southeast Asia':6, 'West Africa':7, 'Southern Europe':8, 'Oceania':9,
 'Northern Europe':10, 'West Asia':11, 'South Asia':12, 'Caribbean':13, 'South of  USA ':14,
 'Eastern Europe':15, 'US Center ':16, 'Eastern Asia':17, 'North Africa':18,
 'Southern Africa':19, 'Central Africa':20, 'Canada':21, 'East Africa':22, 'Central Asia':23}
df['order_region'] = df['order_region'].map(mapping7)
mapping8 = {'COMPLETE':1, 'PENDING':2, 'PROCESSING':3, 'CLOSED':4, 'ON_HOLD':5, 'PENDING_PAYMENT':6,
 'PAYMENT_REVIEW':7}
df['order_status'] = df['order_status'].map(mapping8)
# Функция для определения рабочего дня
def is_workday(date_str):
    date = pd.to_datetime(date_str)
    if date.weekday() < 5:  # Понедельник-Пятница
        return 1
    else:
        return 0

# Применяем функцию к столбцу и сохраняем результат в новом столбце или изменяем текущий
df['order_date'] = df['order_date'].apply(is_workday)

df=df[['payment_type','profit_per_order','sales_per_customer','category_id','customer_country','customer_id','customer_segment','customer_state',
         'customer_zipcode','department_id','latitude','longitude','market','order_customer_id','order_date',
         'order_id','order_item_cardprod_id','order_item_discount','order_item_discount_rate',
         'order_item_id','order_item_product_price','order_item_profit_ratio','order_item_quantity','order_region',
         'sales','order_item_total_amount','order_profit_per_order','product_card_id','product_category_id','order_status',
         'product_price','shipping_mode',"label"]]

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Рассчитываем корреляционную матрицу
correlation_matrix = df.corr(method='pearson')

# Отображаем тепловую карту корреляций
plt.figure(figsize=(32, 12))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Корреляционная матрица признаков")
plt.show()

y=df[['label']]
X=df[['payment_type','profit_per_order','sales_per_customer','category_id','customer_country','customer_id','customer_segment','customer_state',
         'customer_zipcode','department_id','latitude','longitude','market','order_customer_id',
         'order_id','order_item_cardprod_id','order_item_discount','order_item_discount_rate','order_date',
         'order_item_id','order_item_product_price','order_item_profit_ratio','order_item_quantity','order_region',
         'sales','order_item_total_amount','order_profit_per_order','product_card_id','product_category_id','order_status',
         'product_price','shipping_mode']]
y['label'] = y['label'].replace({0: 1, -1: 0})

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создание улучшенной модели нейросети
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    Dense(1, activation='sigmoid')  # Выходной слой для бинарной классификации
])

# Компиляция модели с более низкой скоростью обучения
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Увеличенное количество эпох и размер батча
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Оценка модели на тестовых данных
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Прогноз и отчет о классификации
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
